In [10]:
import psycopg as pg
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

# Load environment variables from .env file
load_dotenv()

DB_CONFIG = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

def get_close(tickers: list[str], start_date: str, end_date: str) -> pd.DataFrame:
    # Convert Python list to SQL-friendly format: ('AAPL', 'MSFT')
    ticker_str = "(" + ", ".join([f"'{ticker}'" for ticker in tickers]) + ")"

    with pg.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cur:
            query = f"""
                SELECT date, ticker, adj_close::float 
                FROM daily
                WHERE ticker IN {ticker_str}
                AND date BETWEEN '{start_date}' AND '{end_date}'
                ORDER BY date
            """
            cur.execute(query)
            data = cur.fetchall()

    # Convert to DataFrame and pivot to get tickers as columns
    df = pd.DataFrame(data, columns=['date', 'ticker', 'adj_close'])
    df['adj_close'] = df['adj_close'].astype(float)  # Ensure adj_close is float
    df = df.pivot(index='date', columns='ticker', values='adj_close')

    return df

In [11]:
tickers = ['AAPL', 'MSFT', 'NVDA']
df = get_close(tickers, '2023-01-01', '2025-01-01')
df[(df == 0).any(axis=1)]

ticker,AAPL,MSFT,NVDA
date,,,


In [12]:
for ticker in df.columns:
    log_rets = np.log(df[ticker] / df[ticker].shift(1))
    print(log_rets)

date
2023-01-03         NaN
2023-01-04    0.010261
2023-01-05   -0.010661
2023-01-06    0.036133
2023-01-09    0.004080
                ...   
2024-12-24    0.011413
2024-12-26    0.003171
2024-12-27   -0.013331
2024-12-30   -0.013352
2024-12-31   -0.007083
Name: AAPL, Length: 502, dtype: float64
date
2023-01-03         NaN
2023-01-04   -0.044729
2023-01-05   -0.030086
2023-01-06    0.011716
2023-01-09    0.009689
                ...   
2024-12-24    0.009330
2024-12-26   -0.002781
2024-12-27   -0.017453
2024-12-30   -0.013328
2024-12-31   -0.007869
Name: MSFT, Length: 502, dtype: float64
date
2023-01-03         NaN
2023-01-04    0.029867
2023-01-05   -0.033366
2023-01-06    0.040797
2023-01-09    0.050458
                ...   
2024-12-24    0.003930
2024-12-26   -0.002070
2024-12-27   -0.021088
2024-12-30    0.003497
2024-12-31   -0.023550
Name: NVDA, Length: 502, dtype: float64


In [9]:
df['AAPL'].astype('float64')

date
2023-01-03    123.63253
2023-01-04    124.90771
2023-01-05    123.58310
2023-01-06    128.13023
2023-01-09    128.65413
                ...    
2024-12-24    257.91644
2024-12-26    258.73550
2024-12-27    255.30930
2024-12-30    251.92302
2024-12-31    250.14497
Name: AAPL, Length: 502, dtype: float64

In [3]:
import psycopg as pg
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np

# Load environment variables from .env file
load_dotenv()

DB_CONFIG = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}


with pg.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        query = """
            SELECT * 
            FROM tickers
        """
        cur.execute(query)
        data = cur.fetchall()

# Convert to DataFrame
df_tickers = pd.DataFrame(data, columns=['ticker', 'sector'])
df_tickers.to_csv('tickers.csv', index=False)